<a href="https://colab.research.google.com/github/sampratha28/Article-Summarizer/blob/main/article_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#problem statement - create a article summarizer to summarise long and complex text

In [ ]:
#two types of summarisation - paragraph, bullet points
#control length of summary generated

#Article Summarizer Streamlit UI

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [3]:
%%writefile app.py

import streamlit as st
from transformers import AutoTokenizer, pipeline, GPTNeoForCausalLM
import torch
import os

offload_dir = "./offload_folder"
os.makedirs(offload_dir, exist_ok=True)

#falcon = tiiuae/falcon-7b-instruct
#distilgpt2
class ArticleSummarizer:
    def __init__(self, model_name="distilgpt2"):
        #Load the model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.text_generator = pipeline(
            "text-generation",
            model=model_name,
            tokenizer=self.tokenizer,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            device_map="auto",
        )




    def generate_summary(self, input_text, summary_format, summary_length):
        # Construct the prompt based on the selected summary length

        if summary_format == 'Bullet Points':
            prompt = f"""Summarize the following text in bullet points, highlighting the most important details and key information. Text:{input_text}\n Summary:
            Example text: In the ancient land of Eldoria, a realm forgotten by time, nestled between the towering peaks of the Seraph Mountains and the whispering depths of the Enchanted Forest, there existed a kingdom unlike any other. This was a land where magic flowed as freely as the rivers that carved their paths through the valleys, where mythical creatures roamed the skies and forests, and where the very earth hummed with an ancient power that echoed the songs of the stars.

For centuries, the Kingdom of Eldoria thrived under the wise and benevolent rule of the Aetherian dynasty. The Aetherians were a noble lineage of rulers who possessed the ability to commune with the elements, drawing strength from the earth, the winds, the waters, and the flames. This connection to the elemental forces granted them wisdom beyond their years and power that was revered by their people. Under their rule, Eldoria became a beacon of peace, prosperity, and harmony, where all beings, whether human, elf, dwarf, or otherwise, lived in unity.

But as with all things in the world, change was inevitable.
            Example Summary: Location: Eldoria, an ancient and mystical land situated between the Seraph Mountains and the Enchanted Forest.
Environment: The kingdom is characterized by abundant magic, mythical creatures, and a land imbued with ancient power.
Ruling Dynasty: The Aetherian dynasty, known for their wisdom and ability to commune with elemental forces (earth, wind, water, fire).
Governance: The Aetherians' rule brought peace, prosperity, and harmony to Eldoria, with unity among all beings (humans, elves, dwarves, etc.).
Current Situation: The text hints at impending change in Eldoria.





"""
        else:
          prompt = f"Summarize the following text in a concise paragraph: {input_text}\n Summary:"

        if summary_length=='Short':
          # Generate the summary using the text generation model
          sequences = self.text_generator(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    top_k=10,  # Only sample from the top 10 tokens
    top_p=0.9,  # Include tokens until 90% of probability mass is covered
    temperature=1.0,  # Default temperature
    num_return_sequences=1,
    no_repeat_ngram_size=2,  # Prevent repetitive sequences
    eos_token_id=self.tokenizer.eos_token_id,
)
        else:
          sequences = self.text_generator(
    prompt,
    max_new_tokens=412,
    do_sample=True,
    top_k=10,  # Only sample from the top 10 tokens
    top_p=0.9,  # Include tokens until 90% of probability mass is covered
    temperature=1.0,  # Default temperature
    num_return_sequences=1,
    no_repeat_ngram_size=2,  # Prevent repetitive sequences
    eos_token_id=self.tokenizer.eos_token_id,
)

        summary = sequences[0]['generated_text'].replace(prompt, "").strip()
        print(summary)

        return summary

# Streamlit app layout
class SummarizerApp:
    def __init__(self):
        self.summarizer = ArticleSummarizer()

    def run(self):
        st.title("Article Summarizer")

        # Text input
        user_input = st.text_area("Enter the text you want to summarize:", height=200)

        # Summary format selection
        summary_format = st.radio(
            "Select the summary format:",
            ('Paragraph', 'Bullet Points')
        )

        # Summary length selection
        summary_length = st.radio(
            "Select the summary length:",
            ('Short', 'Long')
        )

        # Generate and display the summary
        if st.button("Generate Summary"):
            if user_input:
                summary = self.summarizer.generate_summary(user_input, summary_format, summary_length)
                st.subheader("Summary:")
                st.text(summary)
            else:
                st.warning("Please enter some text to summarize.")

# Run the app
if __name__ == "__main__":
    app = SummarizerApp()
    app.run()

Writing app.py


In [4]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [5]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.124.250.230


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.124.250.230:8501

your url is: https://gentle-trains-ask.loca.lt
2025-03-03 16:13:49.704910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741018429.729159    1108 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741018429.736850    1108 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 16:13:49.762030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following i